In [1]:
# LSTM v1 - Simple Lstm experiments

In [66]:
%load_ext autoreload
%autoreload 2

import readers.tac_prep_corpus as tc
from readers.tac_prep_corpus import *

print data_dir

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
readers/../data/


In [67]:
data_tac2014_train = data_dir+"/clear_data/data_tac2014_train.json"
data_tac2014_eval = data_dir+"/clear_data/data_tac2014_eval.json"
data_tac2015_train = data_dir+"/clear_data/data_tac2015_train.json"
data_tac2015_eval = data_dir+"/clear_data/data_tac2015_eval.json"

Vocab and statistics

In [68]:
# reload(readers.tac_prep_corpus)
word_counts = TacPrepJsonCorpus.word_counts_from_jsonfiles(json_files = [data_tac2014_train, data_tac2014_eval, data_tac2015_train])

# print "Top 10 tokens:"
# print word_counts[:10]

In [69]:
# word_counts.sort(key = lambda x:x[1], reverse=True)
print "Vocab size:%s" % len(word_counts)
print "Tokens with freq>=%s : %s" % (5, len( [xx for xx in word_counts if xx[1]>=5]))

Vocab size:20459
Tokens with freq>=5 : 6813


In [70]:
# clear the vocab
vocab = [xx[0] for xx in word_counts]
print vocab[-10:]

[u'toured', u'Merion', u'dependence', u'Advocacy', u'BEFORE', u'Musical', u'Fastow', u'Coach', u'six-party', u'volumes']


In [71]:
print len([xx for xx in vocab if ("/" in xx)or("\\" in xx)])

400


# Load the corpus

In [95]:
max_sent_len = 1000 
max_nr_sent = 100000
max_sent_len=max_sent_len
max_nr_sent=max_nr_sent
update_vocab=True
update_tags=True
unknown_word = "UNKNWN"
unknown_tag = "O"
mapping_file = None
data_x_fieldname = "tokens"
data_y_fieldname = "labels_event"

In [96]:
def count_non_zero_label(test_seq):
    cnt = 0
    for item in test_seq:
        for lbl in item.y:
            if lbl!=0:
                cnt += 1
                # print item.y
                break

    return cnt

In [97]:
tac_corpus = TacPrepJsonCorpus([unknown_word], [unknown_tag])

In [98]:
test_seq, test_seq_meta = tac_corpus.read_sequence_list_tac_json([data_tac2015_eval], max_sent_len=max_sent_len, max_nr_sent=max_nr_sent, 
                         update_vocab=update_vocab, update_tags=update_tags, unknown_word=unknown_word, 
                         unknown_tag=unknown_tag, mapping_file=mapping_file, 
                         data_x_fieldname=data_x_fieldname,
                         data_y_fieldname=data_y_fieldname)

readers/../data//clear_data/data_tac2015_eval.json


In [99]:
print "All sents:%s" % len(test_seq)
print "Non zero labels:%s" % count_non_zero_label(test_seq)
print test_seq[0].x
# print test_seq_meta[0]

All sents:4164
Non zero labels:2646
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 6, 7, 21, 30, 31, 32, 25, 33, 34, 35, 36, 25, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 25, 51, 52, 53, 5, 54, 55, 56, 57, 58, 59, 21, 60, 61]
{u'tokens': [u'regime', u'<img\xa0src="http://www.amnesty.org/sites/impact.amnesty.org/files/imagecache/news-highlight/brazil-amnesty-law%2025.04.12.JPG"/>', u'The', u'efforts', u'of', u'federal', u'prosecutors', u'to', u'initiate', u'criminal', u'investigations', u'into', u'past', u'human', u'rights', u'violations', u'marks', u'a', u'crucial', u'moment', u'in', u'Brazil', u"'s", u'history', u',', u'said', u'Amnesty', u'International', u'after', u'federal', u'prosecutors', u'in', u'S\xe3o', u'Paulo', u'charged', u',', u'on', u'24', u'March', u'2012', u',', u'retired', u'Colonel', u'Carlos', u'Alberto', u'Brilhante', u'Ustra', u'and', u'police', u'chief', u'-LRB-', u'delegado', u'-RRB-', u'D

In [100]:
train_seq, train_seq_meta = tac_corpus.read_sequence_list_tac_json([data_tac2014_train, data_tac2015_train], max_sent_len=max_sent_len, max_nr_sent=max_nr_sent, 
                         update_vocab=update_vocab, update_tags=update_tags, unknown_word=unknown_word, 
                         unknown_tag=unknown_tag, mapping_file=mapping_file, 
                         data_x_fieldname=data_x_fieldname,
                         data_y_fieldname=data_y_fieldname)

readers/../data//clear_data/data_tac2014_train.json
readers/../data//clear_data/data_tac2015_train.json


In [101]:
print "All sents:%s" % len(train_seq)
print "Non zero labels:%s" % count_non_zero_label(train_seq)

All sents:12573
Non zero labels:5367


In [102]:
dev_seq, dev_seq_meta = tac_corpus.read_sequence_list_tac_json([data_tac2014_eval], max_sent_len=max_sent_len, max_nr_sent=max_nr_sent, 
                         update_vocab=update_vocab, update_tags=update_tags, unknown_word=unknown_word, 
                         unknown_tag=unknown_tag, mapping_file=mapping_file, 
                         data_x_fieldname=data_x_fieldname,
                         data_y_fieldname=data_y_fieldname)

readers/../data//clear_data/data_tac2014_eval.json


In [103]:
print "All sents:%s" % len(dev_seq)
print "Non zero labels:%s" % count_non_zero_label(dev_seq)

All sents:9041
Non zero labels:3749


In [104]:
# Redo indices
train_seq, test_seq, dev_seq = tc.compacify(train_seq, test_seq, dev_seq, theano=True)

# Get number of words and tags in the corpus
nr_words = len(train_seq.x_dict)
nr_tags = len(train_seq.y_dict)

In [105]:
print nr_words
print nr_tags

25583
4


# Run the lstm

In [106]:
import lstm_v1 as lstm_v1
import theano
import theano.tensor as T

In [107]:
emb_size=50
hidden_size=50
SEED = 42

lrate = 0.01

nr_iterations = 5
print "nr_words:%s" % nr_words
print "emb_size:%s" % emb_size
print "hidden_size:%s" % hidden_size
print "nr_tags:%s" % nr_tags
print "lrate:%s" % lrate

nr_words:25583
emb_size:50
hidden_size:50
nr_tags:4
lrate:0.01


In [108]:
# Instantiate LSTM
lstm = lstm_v1.LSTM(n_words = nr_words, 
                    n_emb=emb_size, 
                    n_hidd=hidden_size, 
                    n_tags=nr_tags, 
                    seed=SEED)

print lstm

<lstm_v1.LSTM instance at 0x7f7dbe52d680>


In [111]:
lstm.param

[<TensorType(float64, matrix)>,
 <TensorType(float64, matrix)>,
 <TensorType(float64, matrix)>,
 <TensorType(float64, matrix)>,
 <TensorType(float64, matrix)>]

In [ ]:
# Compile prediction and bacth update functions
x = T.ivector('x')
y = T.ivector('y')
lstm_prediction = theano.function([x], T.argmax(lstm._forward(x), 1))
lstm_cost = -T.mean(T.log(lstm._forward(x))[T.arange(y.shape[0]), y])

# Get list of SGD batch update rule for each parameter
lstm_updates = [(par, par - lrate*T.grad(lstm_cost, par)) for par in lstm.param]

# compile
lstm_batch_update = theano.function([x, y], lstm_cost, updates=lstm_updates)

In [ ]:
for i in range(nr_iterations):
    # Training
    cost = 0
    errors = 0
    for n, seq in enumerate(train_seq):
        cost += lstm_batch_update(seq.x, seq.y)
        errors += sum(lstm_prediction(seq.x) != seq.y)
    acc_train = 100*(1-errors*1./nr_words)
    
    print "Epoch %d: Train cost %2.2f Acc %2.2f %%" % (i+1, cost, acc_train),
    # Evaluation:
    errors = 0
    for n, seq in enumerate(dev_seq):
        errors += sum(lstm_prediction(seq.x) != seq.y)
        acc_dev = 100*(1-errors*1./nr_words)
    print " Devel Acc %2.2f %%" % acc_dev
    sys.stdout.flush()
    
print "Done!"